![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICD10_CM.ipynb)

## **Detect Problem with ICD-10**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## **Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

# **🔎 For about models**

📌 **sbiobertresolve_icd10cm**--> *This model maps clinical findings to their corresponding ICD-10-CM code in healthcare records using Entity Resolvers.*

*   Predicted Entities => **Problem**

📌 **sbiobertresolve_icd10cm_augmented** --> *This model maps clinical findings to ICD-10-CM codes using Entity Resolvers. It is augmented with synonyms, which results in having a four times richer vocabulary than the non-augmented version.*

*   Predicted Entities => **Problem** 

📌 **sbiobertresolve_icd10cm_augmented_billable_hcc**--> *This model maps clinical findings to ICD-10-CM codes, it is augmented with synonyms, what results in having a four times richer vocabulary than non-augmented version. It returns 7-digit codes from ICD-10-CM.*

*   Predicted Entities => **Problem** 

📌 **sbiobertresolve_icd10cm_generalised**--> *This model maps clinical findings to ICD-10-CM codes. It predicts ICD-10-CM 3-character codes what, according to ICD-10-CM code structure, represents the general type of the injury or disease.*

*   Predicted Entities => **Problem**

📌 **sbiobertresolve_icd10cm_slim_billable_hcc**--> *This model maps clinical findings to ICD-10-CM codes . It returns the official resolution text within the brackets. *

*   Predicted Entities => **Problem**

📌 **sbiobertresolve_icd10cm_slim_normalized**--> *This model synonyms having low cosine similarity to unnormalized terms are dropped, making the model slim. It also returns the official resolution text within the brackets inside the metadata *

*   Predicted Entities => **Problem**



# **🔎Define Spark NLP pipeline**

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = nlp.NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])

c2doc = nlp.Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

def pipeline(model_name):
  icd_resolver = medical.SentenceEntityResolverModel.pretrained(model_name, "en", "clinical/models") \
        .setInputCols(["ner_chunk", "sbert_embeddings"]) \
        .setOutputCol("icd_code")\
        .setDistanceFunction("EUCLIDEAN")

  resolver_pipeline = Pipeline(
      stages = [
          documentAssembler,
          sentenceDetector,
          tokenizer,
          word_embeddings,
          clinical_ner,
          ner_converter,
          c2doc,
          sbert_embedder,
          icd_resolver
          ])

  data_ner = spark.createDataFrame([[""]]).toDF("text")
  models = resolver_pipeline.fit(data_ner)
  
  light_model = LightPipeline(models)
  return light_model

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
# returns spark df resolution results

def get_codes_from_df(result_df, chunk, output_col, hcc= False):
    
    
    if hcc:
        
        df = result_df.select(F.explode(F.arrays_zip(result_df[chunk].result, 
                                                     result_df[chunk].metadata, 
                                                     result_df[output_col].result, 
                                                     result_df[output_col].metadata)).alias("cols")) \
                      .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                              F.expr("cols['0']").alias("ner_chunk"),
                              F.expr("cols['1']['entity']").alias("entity"), 
                              F.expr("cols['2']").alias("icd10_code"),
                              F.expr("cols['3']['all_k_results']").alias("all_codes"),
                              F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                              F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()



        codes = []
        resolutions = []
        hcc_all = []

        for code, resolution, hcc in zip(df['all_codes'], df['resolutions'], df['hcc_list']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))
            hcc_all.append(hcc.split(":::"))

        df['all_codes'] = codes  
        df['resolutions'] = resolutions
        df['hcc_list'] = hcc_all
        
    else:
                       
        df = result_df.select(F.explode(F.arrays_zip(result_df[chunk].result, 
                                                           result_df[chunk].metadata, 
                                                           result_df[output_col].result, 
                                                           result_df[output_col].metadata)).alias("cols")) \
                                     .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"), 
                                             F.expr("cols['2']").alias(f"{output_col}"),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()



        codes = []
        resolutions = []

        for code, resolution in zip(df['all_codes'], df['resolutions']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))

        df['all_codes'] = codes  
        df['resolutions'] = resolutions
        
    
    return df

# **🔎 "sbiobertresolve_icd10cm" model**

In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
icd_sdf = pipeline("sbiobertresolve_icd10cm").transform(clinical_note_df)

sbiobertresolve_icd10cm download started this may take some time.
[OK!]


In [ ]:
res_pd = get_codes_from_df(icd_sdf, 'ner_chunk', 'icd_code', hcc=False)

In [ ]:
res_pd.head(10)

,sent_id,ner_chunk,entity,icd_code,all_codes,resolutions
0,0,a nonproductive cough,PROBLEM,R0600,"[R0600, M79639, M79609, J949, L299, S27339S, S...","[Dyspnea, unspecified, Pain in unspecified for..."
1,1,right-sided chest pain,PROBLEM,R1011,"[R1011, M79621, M79604, M79631, M25511, M79601...","[Right upper quadrant pain, Pain in right uppe..."
2,1,fever,PROBLEM,A790,"[A790, A78, M041, A689, R502, R509, A921, A779...","[Trench fever, Q fever, Periodic fever syndrom..."
3,2,pericarditis,PROBLEM,I301,"[I301, B3323, I309, I32, I010, I310, I311, I30...","[Infective pericarditis, Viral pericarditis, A..."
4,2,cough,PROBLEM,R05,"[R05, R062, R0601, J384, R067, R51, R0600, R07...","[Cough, Wheezing, Orthopnea, Edema of larynx, ..."
5,2,right-sided chest pain,PROBLEM,R1011,"[R1011, M79621, M79604, M79631, M25511, M79601...","[Right upper quadrant pain, Pain in right uppe..."
6,3,right-sided pleural effusion,PROBLEM,M25411,"[M25411, J810, M25421, H02841, H2141, M25111, ...","[Effusion, right shoulder, Acute pulmonary ede..."


In [ ]:
from sparknlp_display import EntityResolverVisualizer

sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

light_result = pipeline("sbiobertresolve_icd10cm").fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'icd_code',
               document_col='document'
               )

sbiobertresolve_icd10cm download started this may take some time.
[OK!]


# **🔎 "sbiobertresolve_icd10cm_augmented" model**

In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
icd_sdf = pipeline("sbiobertresolve_icd10cm_augmented").transform(clinical_note_df)

sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]


In [ ]:
res_pd = get_codes_from_df(icd_sdf, 'ner_chunk', 'icd_code', hcc=False)

In [ ]:
res_pd.head(10)

,sent_id,ner_chunk,entity,icd_code,all_codes,resolutions
0,0,a nonproductive cough,PROBLEM,R05,"[R05, R059, R292, A379, J988, R098, R0989, R06...","[non-productive cough (finding), cough, unspec..."
1,1,right-sided chest pain,PROBLEM,R0789,"[R0789, R072, R079, M796, R52, I209, M7960, R1...","[right sided chest pain (finding), retrosterna..."
2,1,fever,PROBLEM,P819,"[P819, R508, R509, B338, A938, F681, A689, P81...","[fever, intermittent fever, fever symptoms, os..."
3,2,pericarditis,PROBLEM,I319,"[I319, I301, I308, I318, B3323, A188, I425, I3...","[pericarditis, infectious pericarditis, adhesi..."
4,2,cough,PROBLEM,R05,"[R05, B948, A37, R053, R051, R292]","[cough, persistent cough, whooping cough, chro..."
5,2,right-sided chest pain,PROBLEM,R0789,"[R0789, R072, R079, M796, R52, I209, M7960, R1...","[right sided chest pain (finding), retrosterna..."
6,3,right-sided pleural effusion,PROBLEM,J90,"[J90, P2889, M25411, J91, J869, J918, R849, Q3...","[pleural effusion, bilateral pleural effusion,..."


In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

light_result = pipeline("sbiobertresolve_icd10cm_augmented").fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'icd_code',
               document_col='document'
               )

sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]


# **🔎 "sbiobertresolve_icd10cm_augmented_billable_hcc" model**

In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
icd_sdf = pipeline("sbiobertresolve_icd10cm_augmented_billable_hcc").transform(clinical_note_df)

sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[OK!]


In [ ]:
res_pd = get_codes_from_df(icd_sdf, 'ner_chunk', 'icd_code', hcc=True)

In [ ]:
res_pd.head(10)

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,hcc_list
0,0,a nonproductive cough,PROBLEM,R05,"[R05, R292, A379, J988, R098, R0989, R0600, H5...","[non-productive cough (finding), cough impulse...","[1||0||0, 1||0||0, 0||0||0, 1||0||0, 0||0||0, ..."
1,1,right-sided chest pain,PROBLEM,R0789,"[R0789, R072, R079, M796, R52, I209, M7960, R1...","[right sided chest pain (finding), retrosterna...","[1||0||0, 1||0||0, 1||0||0, 0||0||0, 1||0||0, ..."
2,1,fever,PROBLEM,P819,"[P819, R508, R509, B338, A938, F681, A689, P81...","[fever, intermittent fever, fever symptoms, os...","[1||0||0, 0||0||0, 1||0||0, 1||0||0, 1||0||0, ..."
3,2,pericarditis,PROBLEM,I319,"[I319, I301, I308, I318, B3323, A188, I425, I3...","[pericarditis, infectious pericarditis, adhesi...","[1||0||0, 1||0||0, 1||0||0, 1||0||0, 1||0||0, ..."
4,2,cough,PROBLEM,R05,"[R05, B948, A37, R053, R292, R093]","[cough, persistent cough, whooping cough, chro...","[1||0||0, 1||0||0, 0||0||0, 1||0||0, 1||0||0, ..."
5,2,right-sided chest pain,PROBLEM,R0789,"[R0789, R072, R079, M796, R52, I209, M7960, R1...","[right sided chest pain (finding), retrosterna...","[1||0||0, 1||0||0, 1||0||0, 0||0||0, 1||0||0, ..."
6,3,right-sided pleural effusion,PROBLEM,J90,"[J90, P2889, M25411, J91, J869, J918, R849, Q3...","[pleural effusion, bilateral pleural effusion,...","[1||0||0, 1||0||0, 0||0||0, 0||0||0, 1||1||115..."


In [ ]:
from sparknlp_display import EntityResolverVisualizer

sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

light_result = pipeline("sbiobertresolve_icd10cm_augmented_billable_hcc").fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'icd_code',
               document_col='document'
               )

sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[OK!]


# **🔎 "sbiobertresolve_icd10cm_generalised" model**

In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
icd_sdf = pipeline("sbiobertresolve_icd10cm_generalised").transform(clinical_note_df)

sbiobertresolve_icd10cm_generalised download started this may take some time.
[OK!]


In [ ]:
res_pd = get_codes_from_df(icd_sdf, 'ner_chunk', 'icd_code', hcc=False)

In [ ]:
res_pd.head(10)

,sent_id,ner_chunk,entity,icd_code,all_codes,resolutions
0,0,a nonproductive cough,PROBLEM,R05,"[R05, A37, R09, R06, J98, Y56, R47, H93, J96, ...","[unproductive cough, whooping cough, unspecifi..."
1,1,right-sided chest pain,PROBLEM,R07,"[R07, M79, G89, I99, M25]","[right sided chest pain, right leg pain, chron..."
2,1,fever,PROBLEM,R50,"[R50, A68, A78, A77, J17, A79, B23]","[fever, recurrent fever, acute q fever, bouton..."
3,2,pericarditis,PROBLEM,I31,"[I31, I30, B33, T46, I32, I09]","[pericarditis, infectious pericarditis, viral ..."
4,2,cough,PROBLEM,R05,"[R05, A37, R09]","[cough, whooping cough, respiratory tract cong..."
5,2,right-sided chest pain,PROBLEM,R07,"[R07, M79, G89, I99, M25]","[right sided chest pain, right leg pain, chron..."
6,3,right-sided pleural effusion,PROBLEM,P28,"[P28, M25, H74, J91, I30, J94, J92, S20, R09, ...","[bilateral pleural effusion, effusion, right s..."


In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

light_result = pipeline("sbiobertresolve_icd10cm_generalised").fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'icd_code',
               document_col='document'
               )

sbiobertresolve_icd10cm_generalised download started this may take some time.
[OK!]


# **🔎 "sbiobertresolve_icd10cm_slim_billable_hcc" model**

In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
icd_sdf = pipeline("sbiobertresolve_icd10cm_slim_billable_hcc").transform(clinical_note_df)

sbiobertresolve_icd10cm_slim_billable_hcc download started this may take some time.
[OK!]


In [ ]:
res_pd = get_codes_from_df(icd_sdf, 'ner_chunk', 'icd_code', hcc=True)

In [ ]:
res_pd.head(10)

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,hcc_list
0,0,a nonproductive cough,PROBLEM,R05,"[R05, R05.9, A37.9, R09.89, R06.00, J98.9, R47...","[unproductive cough [Cough], cough, unspecifie...","[0||0||0, 1||0||0, 0||0||0, 1||0||0, 1||0||0, ..."
1,1,right-sided chest pain,PROBLEM,R07.89,"[R07.89, R07.2, R07.9, M79.604, M79.621, M79.6...",[right sided chest pain (finding) [Other chest...,"[1||0||0, 1||0||0, 1||0||0, 1||0||0, 1||0||0, ..."
2,1,fever,PROBLEM,R50.9,"[R50.9, R50.8, A68.9, A68, A78, A77.1, R50.2, ...","[fever [Fever, unspecified], intermittent feve...","[1||0||0, 0||0||0, 1||0||0, 0||0||0, 1||0||0, ..."
3,2,pericarditis,PROBLEM,I31.9,"[I31.9, I30.1, I31.0, B33.23, I31.1, I30, I30....","[pericarditis [Disease of pericardium, unspeci...","[1||0||0, 1||0||0, 1||0||0, 1||0||0, 1||0||0, ..."
4,2,cough,PROBLEM,R05,"[R05, A37, R05.3, R05.1]","[cough [Cough], whooping cough [Whooping cough...","[0||0||0, 0||0||0, 1||0||0, 1||0||0]"
5,2,right-sided chest pain,PROBLEM,R07.89,"[R07.89, R07.2, R07.9, M79.604, M79.621, M79.6...",[right sided chest pain (finding) [Other chest...,"[1||0||0, 1||0||0, 1||0||0, 1||0||0, 1||0||0, ..."
6,3,right-sided pleural effusion,PROBLEM,P28.89,"[P28.89, M25.41, M25.411, H74.8X1, J91.8, I30....",[bilateral pleural effusion [Other specified r...,"[1||0||0, 0||0||0, 1||0||0, 1||0||0, 1||0||0, ..."


In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

light_result = pipeline("sbiobertresolve_icd10cm_slim_billable_hcc").fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'icd_code',
               document_col='document'
               )

sbiobertresolve_icd10cm_slim_billable_hcc download started this may take some time.
[OK!]


# **🔎 "sbiobertresolve_icd10cm_slim_normalized" model**

In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
icd_sdf = pipeline("sbiobertresolve_icd10cm_slim_normalized").transform(clinical_note_df)

sbiobertresolve_icd10cm_slim_normalized download started this may take some time.
[OK!]


In [ ]:
res_pd = get_codes_from_df(icd_sdf, 'ner_chunk', 'icd_code', hcc=False)

In [ ]:
res_pd.head(10)

,sent_id,ner_chunk,entity,icd_code,all_codes,resolutions
0,0,a nonproductive cough,PROBLEM,R05,"[R05, R05.9, A37.9, R09.89, R06.00, J98.9, R47...","[unproductive cough [Cough], cough, unspecifie..."
1,1,right-sided chest pain,PROBLEM,R07.89,"[R07.89, R07.2, R07.9, M79.604, M79.621, M79.6...",[right sided chest pain (finding) [Other chest...
2,1,fever,PROBLEM,R50.9,"[R50.9, R50.8, A68.9, A68, A78, A77.1, R50.2, ...","[fever [Fever, unspecified], intermittent feve..."
3,2,pericarditis,PROBLEM,I31.9,"[I31.9, I30.1, I31.0, B33.23, I31.1, I30.9, I3...","[pericarditis [Disease of pericardium, unspeci..."
4,2,cough,PROBLEM,R05,"[R05, A37, R05.3, R05.1]","[cough [Cough], whooping cough [Whooping cough..."
5,2,right-sided chest pain,PROBLEM,R07.89,"[R07.89, R07.2, R07.9, M79.604, M79.621, M79.6...",[right sided chest pain (finding) [Other chest...
6,3,right-sided pleural effusion,PROBLEM,P28.89,"[P28.89, M25.41, M25.411, H74.8X1, J91.8, I30....",[bilateral pleural effusion [Other specified r...


In [ ]:
sample_text = """The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion."""

light_result = pipeline("sbiobertresolve_icd10cm_slim_normalized").fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'icd_code',
               document_col='document'
               )

sbiobertresolve_icd10cm_slim_normalized download started this may take some time.
[OK!]
